# Post Trade Analysis

Note: Please drag the SQLite file to the folder where you ran this notebook.

# Plot Candlesticks and Trades


In [ ]:
# Enable autoreload extension
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from analysis.scripts.post_trade_plot import PostTradePlot

tool = PostTradePlot("/tmp/replay.sqlite")
pd.DataFrame.from_dict(tool.get_profit(), orient="index")

In [ ]:
tool.draw()

In [ ]:
tool.load_opportunities()